<h2>Model Inference</h2>

Now we will begin the actual processing.

Simply run the cells and provide the required information in the menu.

Parameter Selection:

<b>YAML Path</b>:<br> 
Path to the inference configuration YAML file for prediction. You can choose from:<br><br>
<b>semantic_seg_2d_inference_2class.yaml</b> — for the previous classic model and its retraining.<br><br>
<b>probabilistic_semantic_seg2D_inference_2Class.yaml</b> — for the new probabilistic model.<br><br>

<b>CKPT Path</b>:<br> Path to the model weights file corresponding to the chosen model:<br>

<b>ClassicUnet_16_06_2025.ckpt</b> — for the retrained classic model.<br><br>

<b>ProbUnet_16_06_2025.ckpt</b> — for the new probabilistic model..<br><br>

These .ckpt files are available for download at the following link: [LINK](https://ambiomcloud.isas.de/index.php/s/CwcfFRt8eQ9gKWj).

 <b>Input base path</b>:<br>  Path to the input files. This should correspond to the out_path_base defined in the data wrangling notebook.    

In [1]:
from core.inference import create_inference_menu
create_inference_menu()

Text(value='./semantic_seg_2d_inference_2class.yaml', description='YAML Path:', layout=Layout(width='80%'), pl…

Text(value='./version_2023_06.ckpt', description='CKPT Path:', layout=Layout(width='80%'), placeholder='Weight…

Text(value='/path/to/splitted/3d/files', description='Input base path:', layout=Layout(width='80%'), placehold…

Button(button_style='success', description='Run Inference', style=ButtonStyle())

Output()